In [1]:
import pandas as pd
from sklearn.datasets import load_iris


In [2]:
iris = load_iris()


In [19]:
from sklearn.datasets import load_digits
digits = load_digits()

In [20]:
X = digits.data
y = digits.target

In [3]:
X = iris.data
y = iris.target

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=5)

In [22]:
import sklearn.svm as svm

model = svm.SVC(kernel='rbf',C=30,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test, y_test)

0.3962962962962963

In [23]:
from sklearn.model_selection import cross_val_score

cross_val_score(svm.SVC(kernel='linear', C=10), X, y, cv = 5)


array([0.96388889, 0.91944444, 0.96657382, 0.9637883 , 0.92479109])

In [24]:
cross_val_score(svm.SVC(kernel='rbf', C=10), X, y, cv = 5)

array([0.98055556, 0.95833333, 0.98328691, 0.98885794, 0.95821727])

In [25]:
cross_val_score(svm.SVC(kernel='linear', C=20), X, y, cv = 5)

array([0.96388889, 0.91944444, 0.96657382, 0.9637883 , 0.92479109])

In [26]:
import numpy as np

kernels = ['rbf', 'linear']
C = [1,10,20]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval,C=cval,gamma='auto'),X, y, cv=5)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)
avg_scores


{'rbf_1': 0.448545341999381,
 'rbf_10': 0.47636645001547506,
 'rbf_20': 0.47636645001547506,
 'linear_1': 0.9476973073351903,
 'linear_10': 0.9476973073351903,
 'linear_20': 0.9476973073351903}

In [28]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(svm.SVC(gamma='auto'), {'C': [1,10,20], 'kernel': ['rbf','linear']}, cv=5, return_train_score=False)
clf.fit(X, y)

clf.cv_results_

df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.222537,0.003034,0.054054,0.000425,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.411111,0.450000,0.454039,0.448468,0.479109,0.448545,0.021761,6
1,0.021506,0.001185,0.005118,0.000058,1,linear,"{'C': 1, 'kernel': 'linear'}",0.963889,0.919444,0.966574,0.963788,0.924791,0.947697,0.020978,1
2,0.222893,0.006026,0.055337,0.002456,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.452778,0.469444,0.479109,0.479109,0.501393,0.476366,0.015784,4
3,0.022582,0.000789,0.005345,0.000409,10,linear,"{'C': 10, 'kernel': 'linear'}",0.963889,0.919444,0.966574,0.963788,0.924791,0.947697,0.020978,1
4,0.233813,0.008519,0.057314,0.001399,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.452778,0.469444,0.479109,0.479109,0.501393,0.476366,0.015784,4
5,0.023401,0.002059,0.005434,0.000465,20,linear,"{'C': 20, 'kernel': 'linear'}",0.963889,0.919444,0.966574,0.963788,0.924791,0.947697,0.020978,1


In [29]:
clf.best_params_

{'C': 1, 'kernel': 'linear'}

In [30]:
clf.best_score_

0.9476973073351903

In [31]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
model_params = {
 'svm': {
 'model': svm.SVC(gamma='auto'),
 'params' : {
 'C': [1,10,20],
 'kernel': ['rbf','linear']
 }
 },
 'random_forest': {
 'model': RandomForestClassifier(),
 'params' : {
 'n_estimators': [1,5,10]
 }
 },
 'logistic_regression' : {
 'model': LogisticRegression(solver='liblinear',multi_class='auto'),
 'params': {
 'C': [1,5,10]
 }
 }
}

In [33]:
scores = []
for model_name, mp in model_params.items():
 clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
 clf.fit(X, y)
 scores.append({
 'model': model_name,
 'best_score': clf.best_score_,
 'best_params': clf.best_params_
 })

df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.947697,"{'C': 1, 'kernel': 'linear'}"
1,random_forest,0.888743,{'n_estimators': 10}
2,logistic_regression,0.922114,{'C': 1}
